In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [3]:
df= pd.read_json("Automotive_5.json",lines = True)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B00002243X,"[4, 4]",5,I needed a set of jumper cables for my new car...,"08 17, 2011",A3F73SC1LY51OO,Alan Montgomery,Work Well - Should Have Bought Longer Ones,1313539200
1,B00002243X,"[1, 1]",4,"These long cables work fine for my truck, but ...","09 4, 2011",A20S66SKYXULG2,alphonse,Okay long cables,1315094400
2,B00002243X,"[0, 0]",5,Can't comment much on these since they have no...,"07 25, 2013",A2I8LFSN2IS5EO,Chris,Looks and feels heavy Duty,1374710400
3,B00002243X,"[19, 19]",5,I absolutley love Amazon!!! For the price of ...,"12 21, 2010",A3GT2EWQSO45ZG,DeusEx,Excellent choice for Jumper Cables!!!,1292889600
4,B00002243X,"[0, 0]",5,I purchased the 12' feet long cable set and th...,"07 4, 2012",A3ESWJPAVRPWB4,E. Hernandez,"Excellent, High Quality Starter Cables",1341360000


In [4]:
df = df[['asin','overall','reviewerID']].copy()
df.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [5]:
from sklearn.model_selection import train_test_split
# Splitting train and test data at the start
df_train, df_test = train_test_split(df,test_size=0.2,stratify = df['reviewerID'])
df_train.head()

,asin,overall,reviewerID
600,B0002F9YHI,4,AMSLN8SSNK1IR
16188,B00480FHG0,5,A1MLVQ20U6X3ZC
10064,B0015YBYCS,2,AZB9A8U06Y8JY
13114,B002BC4N5I,5,A1E5I7Z7Y7HY76
6315,B000E8V9RG,5,AG7MBYH38IN1G


In [6]:
df_train = df_train.sort_index(axis=0)
df_train.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [7]:
df_original_train = df_train.copy()
df_train = df_train.pivot(index = "asin" , columns = "reviewerID" , values = "overall")
df_train = df_train.sort_index(axis=1)
df_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00002243Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008BKX5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
X = df_train.values
Y = df_train.sum(axis = 1)
Z =df_train.count(axis =1).astype('float64')
Y = Y.div(Z,axis=0)
cosine
df_new_train = df_train.sub(Y,axis=0).fillna(0)
df_new_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002243Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008BKX5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
nbrs_train = NearestNeighbors(n_neighbors= 10 , algorithm='auto',metric ='cosine').fit(df_new_train.values)
distances, indices = nbrs_train.kneighbors(df_new_train,return_distance=True)

In [10]:
distances

array([[0.        , 0.91800799, 0.94023857, ..., 0.95312663, 0.96015905,
        0.96037856],
       [0.        , 0.82361658, 0.87752551, ..., 0.90393819, 0.90410734,
        0.91548457],
       [0.        , 0.66666667, 0.95917517, ..., 0.98220454, 0.98285014,
        1.        ],
       ...,
       [0.        , 0.67340137, 0.7763932 , ..., 0.95      , 0.96273616,
        0.96629001],
       [0.        , 0.59910814, 0.74844235, ..., 0.97412901, 0.99494601,
        1.        ],
       [0.        , 0.63726187, 0.77008635, ..., 0.91230919, 0.92975606,
        0.92975606]])

In [11]:
indices

array([[   0,  938, 1127, ...,  849,  437,  185],
       [   1, 1212,  887, ...,   79, 1111,  698],
       [   2, 1228,  756, ...,  848,  412, 1223],
       ...,
       [1832, 1604, 1798, ..., 1072, 1752, 1501],
       [1833, 1173,  169, ..., 1656, 1686, 1221],
       [1834,  618, 1378, ..., 1529, 1751, 1679]])

In [12]:
asin_list = df_new_train.index.values.tolist()
index_list = indices.tolist()
for i in range(5):
    string = str(asin_list[i]) + " : "
    for x in index_list[i][1:]:
            string += asin_list[x] + " , "
    print(string)

B00002243X : B001D8XA50 , B0027VFZE6 , B0024E6YWS , B00042JSNY , B000EDOT82 , B001TE0IMG , B00139YMUQ , B000C5GAQQ , B0006IX850 , 
B00002243Z : B002SIR91A , B0016LA5O8 , B001V8U12M , B00068XCQU , B004P91FCO , B0002SRH7O , B0002NYDZ8 , B0024E6UE0 , B000MY3ML8 , 
B00008BKX5 : B002X6VXL4 , B000RN04RO , B0002H335A , B005ASY23I , B001AWU6KQ , B001EOV492 , B0013935FY , B000BZG29W , B002VVC4TG , 
B00008RW9U : B00167Y3ZO , B001MXTOKQ , B00D8DR0AO , B0033A5PPO , B005KDOA9Y , B000E9TS6O , B00029WT08 , B009OBW28E , B0000AY3X0 , 
B00008RW9V : B002WQHY9K , B002UMC1HQ , B002UX1T78 , B002VVC4TG , B002VVIC06 , B002UD0GGS , B002XL2IEA , B002X6VXL4 , B002UD0GFE , 


In [13]:
item_avg = df_original_train.groupby(['asin']).mean()
user_avg = df_original_train.groupby(['reviewerID']).mean()
mean_avg = df_original_train["overall"].mean()
item_map = df_new_train.index.values

In [14]:
item_avg.head()

,overall
asin,
B00002243X,4.777778
B00002243Z,4.400000
B00008BKX5,4.600000
B00008RW9U,4.500000
B00008RW9V,5.000000


In [15]:
df_asin = df.groupby('asin').count()
df_asin = df_asin.index.values

In [16]:
df_asin

array(['B00002243X', 'B00002243Z', 'B00008BKX5', ..., 'B00JGM7Z5Q',
       'B00JGP8I1S', 'B00KIAQ8VW'], dtype=object)

In [17]:
df_reviewerID = df.groupby('reviewerID').count()
df_reviewerID = df_reviewerID.index.values

In [18]:
df_reviewerID

array(['A00473363TJ8YSZ3YAGG9', 'A0473259F6GQNBD88IYN',
       'A09567722SXKYZTV7OFCJ', ..., 'AZTAK8PFB0NLV', 'AZV2U6GU5QA6C',
       'AZYBZEVISD0YX'], dtype=object)

In [19]:
df_dict = {}
for line in df_original_train.values:
    if line[0] in df_dict:
        df_dict[line[0]].append(line[2])
    else:
        df_dict[line[0]] = [line[2]]

In [20]:
predict_for_this = []
count = 0
for i in range(len(df_reviewerID)):
    for j in range(len(df_asin)):
#         print(df_asin[i])
        if df_asin[j] not  in  df_dict or not df_reviewerID[i] in df_dict[df_asin[j]]:
            predict_for_this.append((df_reviewerID[i],df_asin[j]))

In [32]:
final_prediction = {}

for i in range(10000):
#__________________________________________________________________________________________________________
# for i in range(len(predict_for_this)): This loop needs to be run instead of above one. 5352407 records: takes time
#__________________________________________________________________________________________________________
    
    item_asin = predict_for_this[i][1]
#     print(item_asin)
    item_userID = predict_for_this[i][0]
#     print(item_asin,item_userID)
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
#     print(baseline_estimate)
    
    numerator = 0 
    denomenator = 0
    
    if bi != 0 :
        
        item  = np.where(item_map == item_asin)[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
        cnt = 0;
        k = 0
        
        for j in item_nbrs:
            temp = df_original_train.loc[(df_original_train['asin'] == item_map[j[0]]) &  (df_original_train['reviewerID'] == item_userID)]
            if(temp.size != 0):
                dist = dist_nbrs[cnt:cnt+1]
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 

    if item_userID in final_prediction:
        final_prediction[item_userID].append((item_asin,predicted_rating))
    else:
        final_prediction[(item_userID)] = [(item_asin,predicted_rating)]

#___________________________________________________________
#        Sort final_prediction on basis of rating
#___________________________________________________________

In [33]:
user_list = []
for x in final_prediction:
    user_list.append(x)

In [34]:
#final_prediction_new = {}
for x in final_prediction:
    final_prediction[x] = sorted(final_prediction[x],key = lambda x:x[1],reverse= True)[:10]

In [35]:
final_prediction

{'A00473363TJ8YSZ3YAGG9': [('B00008RW9V', 4.529063377701795),
  ('B00009RAX7', 4.529063377701795),
  ('B00009W3G7', 4.529063377701795),
  ('B0000ATZD5', 4.529063377701795),
  ('B0000AXTUY', 4.529063377701795),
  ('B0000AY60S', 4.529063377701795),
  ('B0000AYES4', 4.529063377701795),
  ('B0000UUX34', 4.529063377701795),
  ('B00020CAVA', 4.529063377701795),
  ('B000221M94', 4.529063377701795)],
 'A0473259F6GQNBD88IYN': [('B00008RW9V', 4.97350782214624),
  ('B00009RAX7', 4.97350782214624),
  ('B00009W3G7', 4.97350782214624),
  ('B0000ATZD5', 4.97350782214624),
  ('B0000AXTUY', 4.97350782214624),
  ('B0000AY60S', 4.97350782214624),
  ('B0000AYES4', 4.97350782214624),
  ('B0000UUX34', 4.97350782214624),
  ('B00020CAVA', 4.97350782214624),
  ('B000221M94', 4.97350782214624)],
 'A09567722SXKYZTV7OFCJ': [('B00008RW9V', 5.529063377701795),
  ('B00009RAX7', 5.529063377701795),
  ('B00009W3G7', 5.529063377701795),
  ('B0000ATZD5', 5.529063377701795),
  ('B0000AXTUY', 5.529063377701795),
  ('B0000

In [36]:
user_list

['A00473363TJ8YSZ3YAGG9',
 'A0473259F6GQNBD88IYN',
 'A09567722SXKYZTV7OFCJ',
 'A10063PJ5C9WQQ',
 'A100UD67AHFODS',
 'A100WO06OQR8BQ']

In [37]:
precision_count = 0
recall_count = 0
avg_precision = 0
avg_recall = 0
for i in user_list:
    precision_count = 0
    recall_count = 0
    for j in range(10):
        #print("hiii")
        if final_prediction[i][j][0] in df_test['asin'].values:
            precision_count += 1
            #print(final_prediction[i][j][0])
        if final_prediction[i][j][0] in (df_test.loc[df_test['reviewerID']== i]):
            recall_count += 1
            #print(final_prediction[i][j][0])
                                                    
    user_precision = precision_count/10
    avg_precision+=user_precision
    user_recall = recall_count/(len(df_test.loc[df_test['reviewerID'] == i]))
    avg_recall+= user_recall
pr = avg_precision/(len(user_list))
re = avg_recall/(len(user_list))
print("Precision: ")
print(pr)
print("Recall: ")
print(re)
print("F-measure: ")
print(2*(pr)*(re)/(pr+re))

Precision: 
0.7999999999999999
Recall: 
0.0
F-measure: 
0.0


In [31]:
for i in user_list:
    print(df_test.loc[df_test['reviewerID']== i])

            asin  overall             reviewerID
6673  B000EH4V18        5  A00473363TJ8YSZ3YAGG9
            asin  overall            reviewerID
3242  B0009IQXJ0        5  A0473259F6GQNBD88IYN
3669  B0009VIQ1A        5  A0473259F6GQNBD88IYN
             asin  overall             reviewerID
10430  B001AWLTX4        5  A09567722SXKYZTV7OFCJ
             asin  overall      reviewerID
15530  B003V9L05G        5  A10063PJ5C9WQQ
            asin  overall      reviewerID
8756  B000SM6OD2        5  A100UD67AHFODS
             asin  overall      reviewerID
4443   B000BO22NS        5  A100WO06OQR8BQ
11630  B001NQF3QQ        1  A100WO06OQR8BQ
6616   B000EDUUIA        4  A100WO06OQR8BQ
